In [19]:
import numpy as np
import desurvey
import composite
import dask.dataframe as dd
import pandas as pd

In [36]:
assay = pd.read_csv("../data/assay.csv")
survey =pd.read_csv("../data/survey.csv")
collar =pd.read_csv("../data/collar.csv")

In [37]:
assay = dd.from_pandas(assay, npartitions=2)

In [5]:
assay.head()

,DHID,FROM,TO,_len,DUM,_acum,azmm,dipm,xm,ym,...,azme,dipe,xe,ye,ze,dist_hw,dist_fw,D1_surf,D1_solid,Au
0,0,0.0,1.0,1.0,0.0,0.0,90.0,86.774025,4.561190,-1.229899e-09,...,90.0,86.774147,4.589327,-2.459786e-09,98.728762,-1.0,-1.0,-1.0,0,0.0
1,0,1.0,2.0,1.0,0.0,0.0,90.0,86.774147,4.617463,-3.689650e-09,...,90.0,86.774086,4.645599,-4.919526e-09,97.730346,-1.0,-1.0,-1.0,0,0.0
2,0,2.0,3.0,1.0,0.0,0.0,90.0,86.774086,4.673736,-6.149413e-09,...,90.0,86.774025,4.701872,-7.379312e-09,96.731931,-1.0,-1.0,-1.0,0,0.0
3,0,3.0,4.0,1.0,0.0,0.0,90.0,86.774086,4.730009,-8.609211e-09,...,90.0,86.774086,4.758146,-9.839098e-09,95.733516,-1.0,-1.0,-1.0,0,0.0
4,0,4.0,5.0,1.0,0.0,0.0,90.0,86.774086,4.786282,-1.106898e-08,...,90.0,86.774086,4.814419,-1.229887e-08,94.735100,-1.0,-1.0,-1.0,0,0.0


In [6]:
survey.head()

,DHID,DEPTH,DIP,AZIMUTH
0,0,0.000000,86.774086,90.0
1,0,5.653160,86.774086,90.0
2,0,8.479740,85.789413,90.0
3,0,11.306320,84.804726,90.0
4,0,14.132899,83.820038,90.0


In [7]:
collar.head()

,DHID,X,Y,Z,LENGTH
0,0,4.533054,0.0,99.727177,79.144237
1,1,35.592865,0.0,112.151102,97.982238
2,10,158.321091,50.0,105.771249,53.218356
3,11,5.372508,100.0,89.989507,77.448597
4,12,37.271773,100.0,100.062959,71.202439


In [7]:
def get_above_below_idx(arr, val):
    above = np.where(arr<val)[0][-1]
    below = np.where(arr>val)[0][0]
    return above, below

In [8]:
def get_survey_interp(assay_depth, dh_survey):
    pritn(assay_depth)
    a_from, b_from = get_above_below_idx(dh_survey.DEPTH, assay_depth)
    length = dh_survey.DEPTH[b_from] - dh_survey.DEPTH[a_from]
    dist = assay_depth - dh_survey.DEPTH[a_from]
    az, dip = desurvey.interp_survey(dh_survey[a_from].AZIMUTH, dh_survey[a_from].DIP, dh_survey[b_from].AZIMUTH, dh_survey[b_from].DIP, length, dist)
    return az, dip



In [9]:
assay_dh_groups = assay.groupby("DHID")
survey_dh_groups = survey.groupby("DHID")

In [95]:
assay["midpt"] = (assay.FROM + assay.TO)/2

In [96]:
def get_above_below_idx(arr, val):
    above = np.where(arr<val)[0][-1]
    below = np.where(arr>val)[0][0]
    return above, below

In [188]:
def get_survey_interp(row, survey):
    dhid = row.DHID
    assay_depth = row.midpt
    dh_survey = survey[survey.DHID == dhid].reset_index()
    a_from, b_from = get_above_below_idx(dh_survey.DEPTH, assay_depth)
    length = dh_survey.DEPTH[b_from] - dh_survey.DEPTH[a_from]
    dist = assay_depth - dh_survey.DEPTH[a_from]
    top = survey.iloc[a_from]
    bottom = survey.iloc[b_from]

    az, dip = desurvey.interp_survey(top.AZIMUTH, top.DIP,        bottom.AZIMUTH, bottom.DIP, length, dist)
    return int(dhid), az, dip, assay_depth

In [189]:
interp = assay.apply(get_survey_interp, args=(survey,), axis=1, result_type="expand").compute()

In [194]:
interp.rename(columns=dict(zip(interp.columns,["DHID","AZIMUTH","DIP", "DEPTH"])), inplace=True)

In [195]:
interp

,DHID,AZIMUTH,DIP,DEPTH
0,0.0,90.0,86.774086,0.5
1,0.0,90.0,86.774086,1.5
2,0.0,90.0,86.774086,2.5
3,0.0,90.0,86.774086,3.5
4,0.0,90.0,86.774086,4.5
...,...,...,...,...
1877,9.0,90.0,82.366471,54.5
1878,9.0,90.0,82.250275,55.5
1879,9.0,90.0,82.135786,56.5
1880,9.0,90.0,82.022930,57.5


In [196]:
desurvey.desurvey(interp, collar)

,dhid,x,y,z
0,0.0,4.533054,0.000000e+00,99.727177
1,0.0,0.056273,3.445732e-18,98.728762
2,0.0,0.056273,3.445732e-18,97.730347
3,0.0,0.056273,3.445732e-18,96.731931
4,0.0,0.056273,3.445732e-18,95.733516
...,...,...,...,...
1877,9.0,0.131816,8.071394e-18,48.785714
1878,9.0,0.133841,8.195418e-18,47.794712
1879,9.0,0.135836,8.317560e-18,46.803981
1880,9.0,0.137801,8.437900e-18,45.813521
